In [ ]:
#OS level tools
import os
import time
import itertools
from collections import defaultdict
from glob import glob
import psutil
from functools import partial
import re

#array and data structure
import numpy as np
import pandas as pd

#Statistical analysis
#import seaborn as sb
#from sklearn.metrics import f1_score

#Ipython display and widgets
import ipywidgets as widgets
from IPython.display import Image, HTML, display
from IPython.display import clear_output
from IPython.display import Markdown as md

#from ipywidgets import interact_manual

#holoviews and plotting
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import aggregate, shade, datashade
#from holoviews.operation import decimate
from matplotlib.cm import viridis, jet

#dask parallelization
#import dask.dataframe as dd
from dask import compute, delayed
import dask.threaded
#import dask.multiprocessing

#color assignment
cmap_fire=['brown', 'maroon', 'red', 'orange', 'yellow', 'white']
cmap_raw=['blue','green','orange', 'red']
cmap_all=['white','white']
cmap_parent=['black','grey']
cmap_pop=(['darkgreen','lightgreen'], ['darkorange','yellow'], ['purple','blueviolet'], ['darkblue','lightblue'], ['indianred','red'])
for i in range(5):
    cmap_pop=cmap_pop+cmap_pop
background = '#D3D3D3'

#export path assignment
#temp_path='/scratch/'+os.environ['USER']+'/'+os.environ['SLURM_JOBID']
export_path="PNG"
#export = partial(export_image, export_path=export_path, background=background)

hv.notebook_extension('bokeh')
display(HTML("<style>.container { width:100% !important;}</style>"))

In [ ]:
hv.opts("RGB [toolbar=None, width=400, height=400, bgcolor='#D3D3D3', fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':5}]")

In [ ]:
def config_objects(s):
    try:
        with open(s) as config_file:
            config_file.seek(0)
            gates={}
            for line in config_file:
                phenoType=""
                line = line.strip()
                gate = line.split("\t")
                if len(gate)==12:
                    phenoType=gate[11]
                gates.update({"pop"+str(gate[0]):[int(gate[0]), int(gate[1]), int(gate[2]), int(gate[3]), int(gate[4]), int(gate[5]), int(gate[6]), int(gate[7]), int(gate[8]), int(gate[9]), int(gate[10]), phenoType]})
            return gates
    except:
        raise Exception("Error parsing configuration file")
        
def config_summary(s, h):
    try:
        with open(s) as config_file:
            config_file.seek(0)
            gates={}
            for line in config_file:
                phenoType=""
                line = line.strip()
                gate = line.split("\t")
                xmarker=str(h[int(gate[1])-1])
                ymarker=str(h[int(gate[2])-1])
                startx=int((float(gate[3])/200)*4096)
                starty=int((float(gate[5])/200)*4096)
                endx=int((float(gate[4])/200)*4096)
                endy=int((float(gate[6])/200)*4096)
                parent="pop"+gate[7]
                if len(gate)==12:
                    phenoType=gate[11]
                gates.update({"pop"+str(gate[0]):[int(gate[0]), parent, xmarker, ymarker, phenoType, startx, endx, starty, endy]})
            return gates
    except:
        raise Exception("Error parsing configuration file")
        
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]   

def natural_sort(l): 
    #https://stackoverflow.com/a/4836734/846892
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text

    return [atoi(c) for c in re.split('(\d+)', text)]

def label_color (pops, row):
    eventcolor=0
    for i, pop in enumerate(pops):
        if row[pop]==0:
            eventcolor=i+1
    return eventcolor

def label_color2 (pops, row):
    eventcolor="base"
    for i, pop in enumerate(pops):
        if row[pop]==0:
            eventcolor=pop
    return eventcolor

def parseCohort(s):
    cohort_file=open(s)
    
    return

def parseDataFrame(s):
    result_file=open(s)
    sampleLabel=os.path.splitext(s)[0]
    events = sum(1 for line in result_file) -1 #quickly determine number of events
    result_file.seek(0) #rewind file to beginning
    header = result_file.readline()
    header = header.strip()
    headers = header.split("\t")
    
    pop_offset=len(headers)
    popList=[]
    for i,header in enumerate(headers):
        if header == "pop1":
            pop_offset=i
        if "pop" in header:
            popList.append(header)
    markers = headers[0:pop_offset]
    result_file.seek(0) #rewind file to beginning
    
    df = pd.read_csv(s, sep='\t')
    dataIndex={}
    for i,header in enumerate(headers):
        dataIndex.update({header:i})
    df['pop0']=0
    return [sampleLabel,headers,markers,popList,df]

def parseDAFi(s):
    df = pd.read_csv(s, sep='\t')
    df['pop0']=0
    return df

def html_row(file):
     return '<img src="{}" style="display:inline;margin:1px" title="{}"/>'.format(export_path+"/"+file+".png",file,file)

# Composite 2D/3D Dot Plots

In [ ]:
titlefilename=glob('description.txt')
if titlefilename:
    titlefile=open(titlefilename[0])
    title=titlefile.readline()
    titlefile.close()
    display(md("# Dataset: %s"%(title)))
else:
    display(md("# Dataset: no description given"))

## Metadata

In [ ]:
%%output backend='bokeh'
%%opts Table [width=1200]
metadatafilename=glob('metadata.txt')
if metadatafilename:
    metadatafile=open(metadatafilename[0])
    metaheader = metadatafile.readline()
    metaheader = metaheader.strip()
    metaheaders = metaheader.split("\t")
    metaDf=pd.read_csv('metadata.txt', sep='\t')
    metaTable=hv.Table(metaDf)
    display(md("## Metadata"))
    display(metaTable)
else:
    display(md("## No metadata info given"))

In [ ]:
gatedFiles=sorted(glob('Gated/*/flock*.txt'))
gatedDelayed=[[(os.path.split(os.path.dirname(fn))[1]),delayed(parseDAFi)(fn)] for fn in gatedFiles]
sample_labels=[os.path.split(os.path.dirname(fn))[1] for fn in gatedFiles]
dfArray=compute(*gatedDelayed, get=dask.threaded.get)

In [ ]:
headers=list(dfArray[0][1])
pop_offset=len(headers)
popList=[]
for i,header in enumerate(headers):
    if header == "pop1":
        pop_offset=i
    if "pop" in header:
        popList.append(header)
markers = headers[0:pop_offset]

## DAFi Configuration

In [ ]:
%%output backend='bokeh'
%%opts Table.gates [width=1200]
%%opts Table.summary [width=1200]
configLabel="pipeline.config"
gates=config_objects(configLabel)
num_gates = len(gates)
summary=config_summary(configLabel, headers)
num_gates = len(summary)

gatesummary = [v for v in summary.values()]
summaryTable=hv.Table(gatesummary,kdims=['Population','Parent','XMarker','YMarker','phenotype','startx', 'endx', 'starty', 'endy'], group='summary', label='Summary')

sortedTable=summaryTable.sort('Population')
sortedTable

## Marker and Axis Tables

In [ ]:
%%output backend='bokeh'
axis_popIndexDict = defaultdict(list)
popBounds={}
axises=[]
composite_axis=0
last_xmarker=""
last_ymarker=""
last_parent=0
gatesconfig=[]
for i in range(len(gates)):
    pop="pop"+str(i+1)
    config=gates.get(pop)
    xmarker=str(headers[config[1]-1])
    ymarker=str(headers[config[2]-1])
    startx=int((float(config[3])/200)*4096)
    starty=int((float(config[5])/200)*4096)
    endx=int((float(config[4])/200)*4096)
    endy=int((float(config[6])/200)*4096)
    parent=int(config[7])
    ctype=int(config[8])
    popBounds.update({pop:[xmarker, ymarker, startx,starty,endx,endy,ctype,"pop"+str(parent)]})
    key="axis"+str(composite_axis).zfill(2) 
    if (xmarker != last_xmarker) or (ymarker != last_ymarker) or (parent != last_parent):
        composite_axis=composite_axis+1
        key="axis"+str(composite_axis).zfill(2)
        axises.append([xmarker,ymarker,key,"pop"+str(parent)])
    axis_popIndexDict[key].append(pop)
    last_xmarker=xmarker
    last_ymarker=ymarker
    last_parent=parent
    gatesconfig.append([pop,xmarker,ymarker,parent])
poplist=natural_sort(popBounds.keys())
num_axises = len(axises)
markerTable=hv.Table(markers,kdims=['Marker'])
axisTable=hv.Table(axises,kdims=['Xmarker','Ymarker','Axis Index', 'Parent Pop'])
axis_popTable=hv.Table(axis_popIndexDict, kdims=['Axis Index'], vdims=['sub populations'])
markerTable+axisTable+axis_popTable.sort('Axis Index')

In [ ]:
hv.notebook_extension('matplotlib')

In [ ]:
%%output backend='bokeh'
%%opts Table.gates [width=len(sample_labels)*100]
%%opts Table.summary [width=len(sample_labels)*100]

batchpercent_df = pd.read_csv('Gated/Batch_percentages.txt', sep='\t', index_col=0)
batchevents_df = pd.read_csv('Gated/Batch_events.txt', sep='\t', index_col=0)
#hv.Layout(hv.Table(batchpercent_df,kdims=sample_labels, group='summary', label='Summary')+hv.Table(batchevents_df,kdims=sample_labels, group='summary', label='Summary')).cols(1)

In [ ]:
centroidFiles=sorted(glob('Gated/*/cent*.txt'))
reclusterset=set(["pop"+os.path.basename(fn).split(".")[0].split("centroids")[1]for fn in centroidFiles])

In [ ]:
hv.opts("RGB [width=600, height=600, bgcolor='#D3D3D3', fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':10}]")

In [ ]:
hv.opts("Points.cent (color='purple' marker='+' size=10)")

In [ ]:
dfMap={}
for k, sample in enumerate(sample_labels):
    dfMap.update({sample:k})

In [ ]:
def outputSampleGates(sample):
    
    filename=export_path+"/"+sample
    hv.output(hv.NdLayout(combineddfPlots[sample,:]).cols(1), backend='matplotlib', size=200, fig='png', filename=filename)
    
    return filename

def outputPopGates(pop):
    
    filename=export_path+"/"+pop
    length=len(combineddfPlots[:,pop])
    hv.output(hv.NdLayout(combineddfPlots[:,pop]).cols(length), backend='matplotlib', size=200, fig='png', filename=filename)
    
    return filename

def displaySampleGates(hmap, sample):
    
    temp=hv.NdLayout(hmap[sample,:]).cols(1)
    
    return temp

def displayPopGates(hmap, pop):
    
    length=len(hmap[:,pop])
    temp=hv.NdLayout(hmap[:,pop]).cols(length)
    
    return temp

In [ ]:
hv.extension('plotly')

# Composite 2D Dot-Plots Gated Populations

In [ ]:
samplesSelect=widgets.SelectMultiple(
    options=sample_labels,
    value=[sample_labels[0]],
    rows=len(sample_labels),
    description='Samples',
    disabled=False, layout=widgets.Layout(width='40%')
)

popsSelect=widgets.SelectMultiple(
    options=poplist,
    value=[poplist[0]],
    rows=len(poplist),
    description='Populations',
    disabled=False, layout=widgets.Layout(width='15%')
)

xmarkerSelect=widgets.RadioButtons(
    options=markers,
    value=markers[0],
    description='Xmarker',
    disabled=False, layout=widgets.Layout(width='15%')
)
ymarkerSelect=widgets.RadioButtons(
    options=markers,
    value=markers[1],
    description='Ymarker',
    disabled=False, layout=widgets.Layout(width='15%')
)
zmarkerSelect=widgets.RadioButtons(
    options=markers,
    value=markers[2],
    description='Zmarker',
    disabled=False, layout=widgets.Layout(width='15%')
)
select_ui=widgets.HBox([samplesSelect,popsSelect,xmarkerSelect,ymarkerSelect,zmarkerSelect])

In [ ]:
size=600

def updateRaw(samples_selected, xaxis, yaxis):
    hv.output(backend="matplotlib")
    hv.opts("RGB [bgcolor='#D3D3D3', fontsize={'title':20, 'xlabel':15, 'ylabel':15, 'ticks':13}]")
    hv.opts("NdLayout [tight=False transpose=False fig_size=200]")
    
    output=""
    
    if (len(samples_selected)>0):
        hmap = hv.HoloMap({(sample): datashade(hv.Points(dfArray[dfMap.get(sample)][1], kdims=[xaxis,yaxis]), width=size, height=size, x_range=(0,4096), y_range=(0,4096), dynamic=False, link_inputs=False, cmap=viridis)
                    for j, sample in enumerate(samples_selected)}, kdims=['Sample'])
        
        output=hv.NdLayout(hmap[list(samples_selected)]).cols(len(samples_selected))
        
    else:
        print("Please select one or more sample(s) and population(s)")
    return output

def update3DRaw(samples_selected, xaxis, yaxis, zaxis):
    hv.output(backend="plotly", size=200)
    hv.opts("Scatter3D [azimuth=40 elevation=20 color_index=2] (size=1 cmap='viridis')")

    output=""
    
    if (len(samples_selected)>0):
        plotArray = hv.HoloMap({(sample): hv.Scatter3D(dfArray[dfMap.get(sample)][1],kdims=[xaxis,yaxis,zaxis])
                    for k, sample in enumerate(sample_labels)}, kdims=['Sample'])
        output=plotArray
    else:
        print("Please select one or more sample(s) and population(s)")
    return output

def update(samples_selected, pops_selected, xaxis, yaxis):
    hv.output(backend="matplotlib")
    hv.opts("RGB [bgcolor='#D3D3D3', fontsize={'title':20, 'xlabel':15, 'ylabel':15, 'ticks':13}]")
    hv.opts("NdLayout [tight=False transpose=False fig_size=200]")
    
    output=""
    
    if (len(pops_selected)>0) & (len(samples_selected)>0):
        hmap = hv.HoloMap({(pop, sample): datashade(hv.Points(dfArray[dfMap.get(sample)][1], kdims=[xaxis,yaxis]), width=size, height=size, x_range=(0,4096), y_range=(0,4096), dynamic=False, link_inputs=False, cmap=cmap_all)*datashade(hv.Points(dfArray[dfMap.get(sample)][1].loc[dfArray[dfMap.get(sample)][1][pop]==0], kdims=[xaxis,yaxis]), width=size, height=size, x_range=(0,4096), y_range=(0,4096), dynamic=False, link_inputs=False, cmap=cmap_pop[poplist.index(pop)])
                    for k, pop in enumerate(pops_selected) for j, sample in enumerate(samples_selected)}, kdims=['Population', 'Sample'])
        
        output=hv.NdLayout(hmap[list(pops_selected),list(samples_selected)]).cols(len(samples_selected))
        
    else:
        print("Please select one or more sample(s) and population(s)")
    return output

def updateOverlap(samples_selected, pops_selected, xaxis, yaxis):
    hv.output(backend="matplotlib")
    hv.opts("RGB [bgcolor='#D3D3D3', fontsize={'title':10, 'xlabel':15, 'ylabel':15, 'ticks':13}]")
    hv.opts("Layout [tight=False sublabel_format=None fig_size=200]")
    hv.opts("NdOverlay [show_title=True fig_size=200]")
    
    output=""
    
    if (len(pops_selected)>0) & (len(samples_selected)>0):
        allmap= datashade(hv.Points(dfArray[dfMap.get(samplesSelect.value[0])][1], kdims=[xaxis,yaxis]), width=size, height=size, x_range=(0,4096), y_range=(0,4096), dynamic=False, link_inputs=False, cmap=cmap_all)

        hmap = hv.HoloMap({(pop, sample): datashade(hv.Points(dfArray[dfMap.get(sample)][1].loc[dfArray[dfMap.get(sample)][1][pop]==0], kdims=[xaxis,yaxis]), width=size, height=size, x_range=(0,4096), y_range=(0,4096), dynamic=False, link_inputs=False, cmap=cmap_pop[poplist.index(pop)])
                    for k, pop in enumerate(pops_selected) for j, sample in enumerate(samples_selected)}, kdims=['Population', 'Sample'])
        overlap=[(allmap*hv.NdOverlay(hmap[list(pops_selected), sample])).relabel(sample+str(pops_selected)) for sample in samples_selected]
        output=hv.Layout(overlap).cols(len(samples_selected))
        
    else:
        print("Please select one or more sample(s) and population(s)")
    return output

## Interactive Custom Cross Sample/Population/Markers Composite Plot Generator

### Generates Raw or DAFi Gated FCS Plots

#### To select multiple samples and populations from the list menu, please holding ctrl or shift key while clicking

In [ ]:
select_ui

In [ ]:
rawgenerate3dButton = widgets.Button(description="Add Raw 3DPlot")
rawgenerate2dButton = widgets.Button(description="Add Raw 2DPlot")
generateButton = widgets.Button(description="Add DAFi 2DPlot")

rawclearButton = widgets.Button(description="Clear Canvas")

def on_rawgenerate2dbutton_clicked(b):
    display(updateRaw(samplesSelect.value, xmarkerSelect.value, ymarkerSelect.value))
def on_rawgenerate3dbutton_clicked(b):
    display(update3DRaw(samplesSelect.value, xmarkerSelect.value, ymarkerSelect.value, zmarkerSelect.value))
def on_generatebutton_clicked(b):
    display(updateOverlap(samplesSelect.value, popsSelect.value, xmarkerSelect.value, ymarkerSelect.value))
def on_rawclearbutton_clicked(b):
    clear_output()
    display(widgets.HBox([rawgenerate2dButton,rawgenerate3dButton,generateButton,rawclearButton]))

rawgenerate3dButton.style.button_color = 'yellow'
rawgenerate2dButton.style.button_color = 'lightgreen'
rawclearButton.style.button_color = 'red'
rawclearButton.style.font_weight = 'Bold'
rawgenerate2dButton.on_click(on_rawgenerate2dbutton_clicked)
rawgenerate3dButton.on_click(on_rawgenerate3dbutton_clicked)
generateButton.style.button_color = 'lightblue'
generateButton.on_click(on_generatebutton_clicked)
rawclearButton.on_click(on_rawclearbutton_clicked)
display(widgets.HBox([rawgenerate2dButton,rawgenerate3dButton,generateButton,rawclearButton]))

generateButton = widgets.Button(description="Generate Plot")



In [ ]:
display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the programming codes."></form>'''))